# Final Project Spark SQL Notebook

### Small Data

### Make all scripts executable

In [1]:
!!chmod a+x ./*/*.py

["chmod: changing permissions of './P1/spark_sql_p1.py': Operation not permitted",
 "chmod: changing permissions of './P2/spark_sql_p2.py': Operation not permitted",
 "chmod: changing permissions of './P3/spark_sql_p3.py': Operation not permitted",
 "chmod: changing permissions of './P4/spark_sql_p4.py': Operation not permitted",
 "chmod: changing permissions of './P5/spark_sql_p5.py': Operation not permitted"]

### Remove all Results

In [2]:
!rm -rf ./*/*.result

### P1

In [ ]:
# %load ./P1/spark_sql_p1.py
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    logRows = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr : Row( state_name = arr[24], site_num = arr[2], state_code = arr[0], county_code = arr[1]))
    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("log")

    query = "SELECT state_name, \
        COUNT(state_name) \
        FROM (SELECT site_num, \
            CONCAT(state_code,county_code) as county, \
            state_name \
            FROM log GROUP BY county, site_num \
        ) \
        GROUP BY state_name \
        ORDER BY rank DESC"

    final = spark.sql(query)

    final.show(30)

    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

In [19]:
!!time python ./P1/spark_sql_p1.py > ./P1/p1.result

['21/12/22 17:43:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '                                                                                ',
 'real\t0m14.592s',
 'user\t0m1.552s',
 'sys\t0m1.200s']

In [20]:
!cat ./P1/p1.result

"expression 'log.`state_name`' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;;\n'Sort ['rank DESC NULLS LAST], true\n+- Aggregate [state_name#3], [state_name#3, count(state_name#3) AS count(state_name)#10L]\n   +- SubqueryAlias `__auto_generated_subquery_name`\n      +- Aggregate [concat(state_code#2, county_code#0), site_num#1], [site_num#1, concat(state_code#2, county_code#0) AS county#8, state_name#3]\n         +- SubqueryAlias `log`\n            +- LogicalRDD [county_code#0, site_num#1, state_code#2, state_name#3], false\n"


### P2

In [ ]:
# %load ./P2/spark_sql_p2.py
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    logRows = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr : Row( State = arr[24], County = arr[25], countyCode = arr[1], Arithmetic_mean = float(arr[16])))
    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("log")

    stateRanksDF = spark.sql("SELECT State, \
        countyCode AS County_Code, \
        County, \
        AVG(Arithmetic_mean) AS Pollutant_levels \
        FROM log GROUP BY State, countyCode, County \
        ORDER BY Pollutant_levels DESC")
    stateRanksDF.show(100)

    sc.stop()
except Exception as err:
    print(err)
    sc.stop()


In [7]:
!!time python ./P2/spark_sql_p2.py > ./P2/p2.result

['21/12/22 17:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '                                                                                ',
 '',
 '[Stage 2:>                                                          (0 + 4) / 4]',
 '[Stage 2:==============>                                            (1 + 3) / 4]',
 '[Stage 2:============================================>              (3 + 1) / 4]',
 '[Stage 3:==========================>                             (93 + 9) / 200]',
 '[Stage 3:======================================>                (139 + 8) / 200]',
 '[Stage 3:===============

In [8]:
!cat ./P2/p2.result

+-----------------+-----------+--------------------+------------------+
|            State|County_Code|              County|  Pollutant_levels|
+-----------------+-----------+--------------------+------------------+
|        Tennessee|        167|              Tipton|            2556.0|
|         New York|        059|              Nassau|              19.0|
|             Ohio|        029|          Columbiana| 7.385690735785953|
|          Montana|        067|                Park| 5.611212121212121|
|Country Of Mexico|        006|     CHIHUAHUA STATE|         4.5121875|
|   North Carolina|        027|            Caldwell| 4.116666666666667|
|       California|        031|               Kings|3.9843770491803276|
|       California|        039|              Madera|            3.7393|
|   North Carolina|        069|            Franklin|3.3499999999999996|
|         Colorado|        059|           Jefferson|              3.07|
|         Michigan|        125|             Oakland| 2.888877848

### P3

In [ ]:
# %load ./P3/spark_sql_p3.py
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    logRows = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr : Row( Year = arr[11][:4], State = arr[24], Arithmetic_mean = float(arr[16])))
    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("log")

    stateRanksDF = spark.sql("SELECT Year, \
        State, \
        AVG(Arithmetic_mean) AS Pollutant_levels \
        FROM log \
        GROUP BY State, Year \
        ORDER BY Year, Pollutant_levels")
    stateRanksDF.show(100)

    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

In [10]:
!!time python ./P3/spark_sql_p3.py > ./P3/p3.result

['21/12/22 17:40:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '[Stage 0:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 '',
 '[Stage 2:>                                                          (0 + 4) / 4]',
 '[Stage 2:==============>                                            (1 + 3) / 4]',
 '[Stage 2:============================================>              (3 + 1) / 4]',
 '[Stage 3:===========>                                            (40 + 8) / 200]',
 '[Stage 3:===============

In [11]:
!cat ./P3/p3.result

+----+--------------------+--------------------+
|Year|               State|    Pollutant_levels|
+----+--------------------+--------------------+
|1990|      Virgin Islands|                 0.0|
|1990|       West Virginia|                 0.0|
|1990|            Oklahoma|                 0.0|
|1990|           Wisconsin|                 0.0|
|1990|              Hawaii|1.970370370370370...|
|1990|              Nevada|4.208000000000000...|
|1990|              Alaska|4.420833333333333...|
|1990|        South Dakota|            5.705E-4|
|1990|          Washington|5.974999999999999E-4|
|1990|             Wyoming|6.045454545454545E-4|
|1990|                Utah|7.970588235294118E-4|
|1990|          New Mexico|8.222222222222222E-4|
|1990|              Oregon|8.596296296296297E-4|
|1990|             Arizona|8.620134228187919E-4|
|1990|               Maine|9.789285714285713E-4|
|1990|            Colorado|0.002162374100719...|
|1990|         Mississippi|0.002666666666666...|
|1990|            Mi

### P4

In [ ]:
# %load ./P4/spark_sql_p4.py
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    # files
    lines_states = sc.textFile('../data/usa_states.csv')
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    
    # file mapping
    logRows_states = lines_states.filter( lambda line : len(line) > 0)    \
                    .zipWithIndex() \
                    .filter( lambda x: x[1] > 0) \
                    .map(lambda x: x[0]) \
                    .map( lambda line: line.split(',')) \
                    .map( lambda arr : Row( stateName = arr[1], minLat = arr[2], maxLat = arr[3], \
                                            minLon = arr[4], maxLon = arr[5]))
    logRows = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: Row( stateName = arr[24], siteNum = arr[2], countyCode = arr[0]+arr[1], lat = float(arr[5]), lon = float(arr[6]) ) )  
    
    # Creates the dataframes and views
    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("log")

    logRows_statesDF = spark.createDataFrame( logRows_states )
    logRows_statesDF.createOrReplaceTempView("log_states")
    
    query = "SELECT log.name\
        FROM (SELECT log.name,\
            \
        )"
    
    query = "SELECT stateName as State, avg(Dist_Monitor_Center) as Avg_Dist_Monitor_Center \
            FROM (SELECT stateName,\
                countyCode,\
                siteNum,\
                sqrt( pow( (lat-((minLat+maxLat)/2))*111, 2) + pow( (lon-((minLon+maxLon)/2))*111, 2) ) as Dist_Monitor_Center\
                FROM (SELECT log.stateName,\
                    log.countyCode,\
                    log.siteNum,\
                    log.lat,\
                    log.lon,\
                    log_states.minLat,\
                    log_states.maxLat,\
                    log_states.minLon,\
                    log_states.maxLon\
                    FROM log\
                    JOIN log_states ON stateName = name\
                ) table1\
                GROUP BY stateName, countyCode, siteNum, lat, lon, minLat, maxLat, minLon, maxLon\
            ) table2\
            GROUP BY stateName\
            ORDER BY Avg_Dist_Monitor_Center DESC"

    finalDF = spark.sql(query)

    
    finalDF.show(100)

    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

In [ ]:
!!time python ./P4/spark_sql_p4.py > ./P4/p4.result

In [ ]:
!cat ./P4/p4.result

### P5

In [ ]:
# %load ./P5/spark_sql_p5.py
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    # files
    lines_states = sc.textFile('../data/usa_states.csv')
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    
    # file mapping
    logRows_states = lines_states.filter( lambda line : len(line) > 0)    \
                    .zipWithIndex() \
                    .filter( lambda x: x[1] > 0) \
                    .map(lambda x: x[0]) \
                    .map( lambda line: line.split(',')) \
                    .map( lambda arr : Row( state = arr[0], name = arr[1], centerLat = (float(arr[2]) + float(arr[3]))/2, \
                                            centerLon = (float(arr[4]) + float(arr[5]))/2))
    logRows = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: Row( name = arr[24], countyCode = arr[1], stateNUM = arr[2], lat = float(arr[5]), lon = float(arr[6]) ) )  
    
    # Creates the dataframes and views
    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("log")

    logRows_statesDF = spark.createDataFrame( logRows_states )
    logRows_statesDF.createOrReplaceTempView("log_states")
    
    # Atribui a cada monitor único o seu quadrante
    MonitorDF = spark.sql("SELECT log.name, log.countyCode, log.stateNum, \
     CASE \
         WHEN log.lat > log_states.centerLat AND log.lon > log_states.centerLon THEN 'NE' \
         WHEN log.lat > log_states.centerLat AND log.lon < log_states.centerLon THEN 'SE' \
         WHEN log.lat < log_states.centerLat AND log.lon < log_states.centerLon THEN 'SW' \
         WHEN log.lat < log_states.centerLat AND log.lon > log_states.centerLon THEN 'NW' \
         ELSE 'Center or Borders' \
     END AS Quadrant \
     FROM log JOIN log_states ON log.name=log_states.name GROUP BY log.name, log.countyCode, log.stateNum, Quadrant")
    MonitorDF.createOrReplaceTempView("Monitor")

    # Conta o Nr. de monitores em cada quadrante por estado
    finalDF = spark.sql("SELECT name AS State, Quadrant, count(*) AS Num_Monitors  FROM Monitor GROUP BY name, Quadrant")
    
    finalDF.show(100)

    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

In [16]:
!!time python ./P5/spark_sql_p5.py > ./P5/p5.result

['21/12/22 17:41:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 2) / 2]',
 '                                                                                ',
 '',
 '[Stage 1:>                                                          (0 + 4) / 4]',
 '[Stage 1:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 '',
 '[Stage 4:>                  (0 + 2) / 2][Stage 5:>                  (0 + 4) / 4]',
 '[Stage 4:===================(2 + 0) / 2][Stage 5:>                  (0 + 4) / 4]',
 '[Stage 5:>         

In [17]:
!cat ./P5/p5.result

+--------------+--------+------------+
|         State|Quadrant|Num_Monitors|
+--------------+--------+------------+
|          Utah|      SE|           6|
|          Utah|      NW|           3|
|          Utah|      SW|           3|
|        Hawaii|      SE|           2|
|        Hawaii|      NE|           2|
|        Hawaii|      NW|           1|
|     Minnesota|      SW|          21|
|     Minnesota|      NE|          11|
|     Minnesota|      NW|          50|
|     Minnesota|      SE|          12|
|          Ohio|      NE|          30|
|          Ohio|      NW|          10|
|          Ohio|      SE|          15|
|          Ohio|      SW|          34|
|      Arkansas|      SE|           3|
|      Arkansas|      SW|           4|
|      Arkansas|      NE|           2|
|      Arkansas|      NW|           1|
|        Oregon|      SW|          12|
|        Oregon|      NE|           3|
|        Oregon|      SE|          15|
|        Oregon|      NW|           1|
|         Texas|      NW|